In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("/Users/nuelcodes/Data-Science-Python/datasets/ecom-expense/Ecom Expense.csv")

In [3]:
df.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


In [4]:
# Creando las variables dummy, tanto para Gender y City Tier
dummy_gender = pd.get_dummies(df["Gender"], prefix="Gender")
dummy_city_tier = pd.get_dummies(df["City Tier"], prefix="City")

In [5]:
dummy_gender.head()

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [6]:
dummy_city_tier.head()

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [7]:
column_names = df.columns.values.tolist()
column_names

['Transaction ID',
 'Age ',
 ' Items ',
 'Monthly Income',
 'Transaction Time',
 'Record',
 'Gender',
 'City Tier',
 'Total Spend']

In [8]:
df_new = df[column_names].join(dummy_gender)
column_names = df_new.columns.values.tolist()
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0


In [9]:
df_new =df_new[column_names].join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


In [10]:
# Lo siguiente es ver como incluir las variables dummy en el modelo lineal y acceder a sus coeficientes
# Variables predictoras
feature_cols = ["Monthly Income", "Transaction Time",
                "Gender_Female", "Gender_Male",
                "City_Tier 1", "City_Tier 2", "City_Tier 3",
                "Record"]

In [11]:
# Conjunto de datos con los cuales vamos a crear el modelo
X = df_new[feature_cols]
# Lo que quiero predecir
Y = df_new["Total Spend"]

In [12]:
# Creacion del modelo lineal
lm = LinearRegression()
lm.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
# Observar los parametros del modelo
print(lm.intercept_) # Corte con el eje de las ordenadas
print(lm.coef_) # Coeficientes

-79.41713030137362
[ 1.47538980e-01  1.54946125e-01 -1.31025013e+02  1.31025013e+02
  7.67643260e+01  5.51389743e+01 -1.31903300e+02  7.72233446e+02]


In [14]:
# Unir las variables predictoras con sus coeficientes
list(zip(feature_cols, lm.coef_))

[('Monthly Income', 0.14753898049205738),
 ('Transaction Time', 0.15494612549589545),
 ('Gender_Female', -131.02501325554567),
 ('Gender_Male', 131.0250132555456),
 ('City_Tier 1', 76.76432601049527),
 ('City_Tier 2', 55.138974309232474),
 ('City_Tier 3', -131.9033003197278),
 ('Record', 772.2334457445648)]

In [15]:
# Si queremos ver que tan bueno es el modelo
lm.score(X, Y)

0.9179923586131016

El modelo puede ser escrito como: 
    Total_Spend = -79.41713030137362 + Monthly Income * 0.14753898049205738 +
                Transaction Time * 0.15494612549589545 + Gender_Female * (-131.02501325554567) +
                Gender_Male * 131.0250132555456 + City_Tier 1 * 76.76432601049527 +
                City_Tier 2 * 55.138974309232474 + City_Tier 3 * (-131.9033003197278) + 
                Record * 772.2334457445648

In [16]:
# Calcular la Desviacion Tipica de los Residuos
df_new["Prediction"] = -79.41713030137362 + df_new['Monthly Income'] * 0.14753898049205738 + df_new['Transaction Time'] * 0.15494612549589545 + df_new['Gender_Female'] * (-131.02501325554567) + df_new['Gender_Male'] * 131.0250132555456 + df_new['City_Tier 1'] * 76.76432601049527 + df_new['City_Tier 2'] * 55.138974309232474 + df_new['City_Tier 3'] * (-131.9033003197278) + df_new['Record'] * 772.2334457445648

In [17]:
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,Prediction
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0,4903.696720
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0,4799.434826
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0,5157.082504
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0,8068.012996
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0,3581.980335


In [18]:
# Calcula la raiz de la suma de los cuadrados de los errores entre el numero de registros menos las variables predictoras menos 1
SSD = np.sum((df_new['Prediction'] - df_new["Total Spend"])**2)
SSD

1517733985.3408163

In [19]:
RSE = np.sqrt(SSD / (len(df_new) - len(feature_cols) - 1))
RSE

803.1318809818165

In [20]:
# Promedio de ventas
sales_mean = np.mean(df_new["Total Spend"])
sales_mean

6163.176415976714

In [21]:
error = RSE / sales_mean
error

0.13031135680294162

## Eliminar variable dummy

In [ ]:
El modelo orginal: 
    Total_Spend = -79.41713030137362 + Monthly Income * 0.14753898049205738 +
                Transaction Time * 0.15494612549589545 + Gender_Female * (-131.02501325554567) +
                Gender_Male * 131.0250132555456 + City_Tier 1 * 76.76432601049527 +
                City_Tier 2 * 55.138974309232474 + City_Tier 3 * (-131.9033003197278) + 
                Record * 772.2334457445648